In [1]:
#r "nuget:Microsoft.Data.Sqlite,8.0.3"
#r "nuget:Skender.Stock.Indicators,2.5.0"
#r "nuget:ccxt,4.2.85"
#r "nuget:ScottPlot,5.0.30"
#r "nuget:ReactiveProperty,9.5.0"
#r "/workspace/src/Common/obj/Debug/net8.0/Common.dll"
#r "/workspace/src/Domain/obj/Debug/net8.0/Domain.dll"
#r "/workspace/src/Infra/obj/Debug/net8.0/Infra.dll"

using Microsoft.Extensions.Logging;
using BotTrade.Infra;
using BotTrade.Infra.Exchanges;
using BotTrade.Domain;
using BotTrade.Domain.Strategies;


Installed Packages ccxt, 4.2.85 Microsoft.Data.Sqlite, 8.0.3 ReactiveProperty, 9.5.0 ScottPlot, 5.0.30 Skender.Stock.Indicators, 2.5.0

Loading extensions from `/root/.nuget/packages/skiasharp/2.88.8/interactive-extensions/dotnet/SkiaSharp.DotNet.Interactive.dll`

In [2]:
var setting = new Setting.Exchange() 
{ 
    Place = ExchangePlace.Binance, 
    Symbol = Symbol.Spot_BTCUSDT 
};


In [3]:
var factory = LoggerFactory.Create(builder => 
{
    builder.AddConsole();
});
var logger = factory.CreateLogger<PastCandleRepository>();
var repository = new PastCandleRepository(setting, logger);

await repository.Fetch();


info: BotTrade.Infra.PastCandleRepository[0]
      前回更新時: 1718324520000
info: BotTrade.Infra.PastCandleRepository[0]
      Fetched since: 2024/06/13 7:53:23 +00:00, count: 10
info: BotTrade.Infra.PastCandleRepository[0]
      データベース更新完了
